In [1]:
import os
import pickle
import re
import time

from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
import json
# import simplejson as json

In [100]:
headers = {'User-Agent': 'Mozilla/5.0'}

In [101]:
snapshot_url = "https://en.wikipedia.org/wiki/Academy_Award_for_Best_Picture"
snapshot_response = requests.get(snapshot_url)
status_code = snapshot_response.status_code

In [102]:
# It's always a good idea to check your status code!
print(f"Response statuse code: {status_code}")

Response statuse code: 200


In [103]:
film_soup = BeautifulSoup(snapshot_response.text, 'html.parser')

In [104]:
print(film_soup.prettify()[:1000])

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-typography-survey-disabled vector-toc-available" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Academy Award for Best Picture - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled 

In [170]:
def parse_winners(star_soup) -> list:
    winner_list = []
    film_jumble = star_soup.select("table:not(.infobox) tr td i b a", href=True)
    for film in film_jumble:
        film_dict = {}
        film_dict['name'] = film.text
        # a = film.select_one("a", href=True)
        film_dict['url'] = "https://en.wikipedia.org" + film['href']
        film_dict['winner'] = True
        winner_list.append(film_dict)
    return winner_list

In [191]:
def parse_losers(star_soup) -> list:
    loser_list = []
    film_jumble = star_soup.select("table:not(.infobox) tbody tr td i:not(:has(b)) a", href=True)
    for film in film_jumble:
        film_dict = {}
        film_dict['name'] = film.text
        # a = film.select_one("a", href=True)
        film_dict['url'] = "https://en.wikipedia.org" + film['href']
        film_dict['winner'] = False
        loser_list.append(film_dict)
    return loser_list

In [192]:
winner_list = parse_winners(film_soup)
loser_list = parse_losers(film_soup)
# star_list.prettify()
winner_list
print(len(winner_list))
print(len(loser_list))

94
592


In [193]:
# print(loser_list)

In [194]:
win_df = pd.DataFrame(winner_list)
lose_df = pd.DataFrame(loser_list)

In [195]:
lose_df

,name,url,winner
0,Learn how and when to remove this template mes...,https://en.wikipedia.org/wiki/Help:Maintenance...,False
1,7th Heaven,https://en.wikipedia.org/wiki/7th_Heaven_(1927...,False
2,The Racket,https://en.wikipedia.org/wiki/The_Racket_(1928...,False
3,Alibi,https://en.wikipedia.org/wiki/Alibi_(1929_film),False
4,Hollywood Revue,https://en.wikipedia.org/wiki/Hollywood_Revue,False
...,...,...,...
587,Green Book,https://en.wikipedia.org/wiki/Green_Book_(film),False
588,Parasite,https://en.wikipedia.org/wiki/Parasite_(2019_f...,False
589,Nomadland,https://en.wikipedia.org/wiki/Nomadland,False
590,CODA,https://en.wikipedia.org/wiki/CODA_(2021_film),False


In [196]:
lose_df.at[590, "winner"] = True
lose_df = lose_df.drop(0)

In [197]:
df = pd.concat([win_df, lose_df], ignore_index=True)

In [206]:
df["url"].nunique()

591

In [208]:
df.sort_values('winner', ascending=False, inplace=True)

# Drop duplicates based on 'url' and keep the first occurrence (which has the highest 'winner')
df.drop_duplicates('url', inplace=True)

# Resetting the index if needed
df.reset_index(drop=True, inplace=True)

print("DataFrame after keeping only the rows with winner=True for each URL:")
print(df)    

DataFrame after keeping only the rows with winner=True for each URL:
                      name                                                url  \
0                    Wings    https://en.wikipedia.org/wiki/Wings_(1927_film)   
1                    Rocky                https://en.wikipedia.org/wiki/Rocky   
2      Shakespeare in Love  https://en.wikipedia.org/wiki/Shakespeare_in_Love   
3                  Titanic  https://en.wikipedia.org/wiki/Titanic_(1997_film)   
4      The English Patient  https://en.wikipedia.org/wiki/The_English_Pati...   
..                     ...                                                ...   
586            The Hustler          https://en.wikipedia.org/wiki/The_Hustler   
587  Judgment at Nuremberg  https://en.wikipedia.org/wiki/Judgment_at_Nure...   
588        The Longest Day  https://en.wikipedia.org/wiki/The_Longest_Day_...   
589          The Music Man  https://en.wikipedia.org/wiki/The_Music_Man_(1...   
590   Mutiny on the Bounty  https://en.w

In [214]:
dflist = df.to_dict(orient="records")

In [219]:
def add_pages(dflist) -> list:
    for film in dflist:
        snapshot_film = requests.get(film["url"], headers=headers)
        # print(snapshot_star.status_code)
        film['film_page'] = BeautifulSoup(snapshot_film.text, 'html.parser')
        time.sleep(1)
    return dflist

In [220]:
start_time = time.time()
# Here is where your scraping function is run
film_pages = add_pages(dflist)
end_time = time.time()
scraping_time = round((end_time-start_time)/60, 2)
print(f"Scraping took {scraping_time} minutes!")
# save_to_pickle(film_pages, pickle_filename)

Scraping took 12.48 minutes!


In [231]:
print(film_pages[590]["film_page"].prettify()[:1000])

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-typography-survey-disabled vector-toc-available" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Mutiny on the Bounty (1962 film) - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disable

In [268]:
i = 0
for item in film_pages:
    # print(item["winner"])
    i += 1
print(i)

591


In [1015]:
def parse_star_pages(film_pages: list) -> list:
    film_list = []
    for film in film_pages:
        filmdic = {}
        filmdic["name"] = film["name"]
        filmdic["url"] = film["url"]
        filmdic["winner"] = film["winner"]
        movie_info = film["film_page"].select(".infobox.vevent tr")
        for header in movie_info:
            headertext = header.select_one("th")
            if headertext:
                if headertext.text[:3] == "Bud":
                    budget_element = header.select_one("td")
                    if budget_element:
                        filmdic["budget"] = budget_element.text
                if headertext.text[:3] == "Rel":
                    year_element = header.select_one("td li span")
                    if year_element:
                        filmdic["year"] = year_element.text[2:6]
                    else:
                        year_element = header.select_one("td span")
                        if year_element:
                            filmdic["year"] = year_element.text[2:6]
                if headertext.text[:3] == "Dis":
                    dist_element = header.select_one("td a")
                    if dist_element:
                        filmdic["distributor"] = dist_element.text
                    else:
                        dist_element = header.select_one("td")
                        if dist_element:
                            filmdic["distributor"] = dist_element.text


            else:
                filmdic["budget"] = float("nan")
                filmdic["year"] = float("nan")
                filmdic["distributor"] = float("nan")
                # if headertext.text[:3] == "Wri":
                #     director_element = header.select_one("a")
                #     if director_element:
                #         filmdic["writer"] = director_element.text
        # if len(star["star_page"].select("li a.ipc-link.ipc-link--base")) > 0 and star["star_page"].select("li a.ipc-link.ipc-link--base")[0].text.isnumeric():
        #     stardic["year_born"] = int(star["star_page"].select("li a.ipc-link.ipc-link--base")[0].text[:4])
        # elif len(star["star_page"].select("li a.ipc-link.ipc-link--base")) > 1 and star["star_page"].select("li a.ipc-link.ipc-link--base")[1].text.isnumeric():
        #     stardic["year_born"] = int(star["star_page"].select("li a.ipc-link.ipc-link--base")[1].text[:4])
        # else:
        #     stardic["year_born"] = "unknown"
        # allcredits = star["credits_page"].select('[id^="'+stardic["primary_role"]+'"]')
        # stardic["first_credit"] = allcredits[-1].select_one("b a").text
        # stardic["year_first_credit"] = int(allcredits[-1].select_one(".year_column").text[2:6])    
        # stardic["num_credits"] = len(allcredits)
        film_list.append(filmdic)
    return film_list 

In [1016]:
film_data_final = []
film_data_final = parse_star_pages(film_pages)
budget_data = film_data_final

In [1017]:
i = 0
for a in film_data_final:
    i += 1
print(i)

591


In [1018]:
desired_movie = next((movie for movie in film_data_final if movie["name"] == "Mutiny on the Bounty"), None)

if desired_movie:
    print(desired_movie)
else:
    print("Movie not found.")

{'name': 'Mutiny on the Bounty', 'url': 'https://en.wikipedia.org/wiki/Mutiny_on_the_Bounty_(1935_film)', 'winner': True, 'budget': '$1.95 million[1]', 'year': '1935', 'distributor': 'Metro-Goldwyn-Mayer'}


In [1019]:
film_data_final = budget_data
for film in film_data_final:
    # print(film["budget"])
    budget_item = film["budget"]
    if pd.isna(budget_item):
        film["budget"] = film["budget"]
    else:
        # Use a regex to find the first occurrence of "$" and extract the rest
        match = re.search(r'\$(.*)', str(budget_item))
        if match:
            budget_item = match.group(1)
        else:
            budget_item = "nan"
        # budget_item = re.sub(r'.*?\$', '', str(budget_item))
        budget_item = re.sub(r'\[.*', '', budget_item)
        budget_item = re.sub(r',', '', budget_item)
        # print(budget_item)
        # print(budget_item)
        million_match = re.search(r'million', str(budget_item))
        first_number_match = re.search(r'\d+(\.\d+)?', budget_item)
        if million_match:
        # Handle ranges like "$25-30 million"
            range_match = re.match(r'(\d+)-(\d+) million', budget_item)
            if range_match:
            # Take the higher number in the estimate range
                budget_item = range_match.group(2)
        if first_number_match:
            first_number = first_number_match.group(0)
            if float(first_number) < 1000:
            # print("This is the first number ", float(first_number))
                budget_item = str(int(float(first_number)* 1000000))
        elif million_match:
            # Assuming you want to convert "$2 million" to "2000000"
            million_value = float(re.sub(r'\D', '', budget_item))  # Extract numerical part
            budget_item = str(int(million_value * 1000000))

        budget_item = re.sub(r'\ .*', '', str(budget_item))
        if budget_item.isdigit():
            film["budget"] = budget_item
        else:
            film["budget"] = "nan"
    # print(film["budget"])

        
        
    

In [1020]:
df = pd.DataFrame(film_data_final)
df["year"].isna().sum()
df = df.rename(columns = {"year": "Year"})

In [1021]:
df.loc[df['name'] == 'The Last Emperor', 'Year'] = 1987
df.loc[df['name'] == 'Hollywood Revue', 'Year'] = 1929
df.loc[df['name'] == 'The Sting', 'Year'] = 1973
df.loc[df['name'] == 'In the Heat of the Night', 'Year'] = 1967
df.loc[df['name'] == 'A Man for All Seasons', 'Year'] = 1966
df.loc[df['name'] == 'Midnight Cowboy', 'Year'] = 1969
df.loc[df['name'] == 'Toy Story 3', 'Year'] = 2010
df.loc[df['name'] == "Precious: Based on the Novel 'Push' by Sapphire", 'Year'] = 2009
df.loc[df['name'] == 'Munich', 'Year'] = 2005
df.loc[df['name'] == 'The Queen', 'Year'] = 2006
df.loc[df['name'] == 'Silver Linings Playbook', 'Year'] = 2012
df.loc[df['name'] == 'Hope and Glory', 'Year'] = 1987
df.loc[df['name'] == 'Sideways', 'Year'] = 2004
df.loc[df['name'] == '49th Parallel', 'Year'] = 1941
df.loc[df['name'] == "King's Row", 'Year'] = 1942
df.loc[df['name'] == 'Random Harvest', 'Year'] = 1942
df.loc[df['name'] == 'Bad Girl', 'Year'] = 1931
df.loc[df['name'] == 'The Big House', 'Year'] = 1930
df.loc[df['name'] == 'The Love Parade', 'Year'] = 1929
df.loc[df['name'] == 'Romeo and Juliet', 'Year'] = 1968
df.loc[df['name'] == 'Midnight Express', 'Year'] = 1978
df.loc[df['name'] == "Coal Miner's Daughter", 'Year'] = 1980
df.loc[df['name'] == 'Father of the Bride', 'Year'] = 1950
df.loc[df['name'] == 'Romeo and Juliet', 'Year'] = 1968
df.loc[df['name'] == 'The Quiet Man', 'Year'] = 1952
df.loc[df['name'] == 'Anatomy of a Murder', 'Year'] = 1959
df.loc[df['name'] == 'The Russians are Coming, the Russians Are Coming', 'Year'] = 1966
df.loc[df['name'] == 'Elmer Gantry', 'Year'] = 1960
df.loc[df['name'] == 'To Kill a Mockingbird', 'Year'] = 1962
df.loc[df['name'] == 'Mutiny on the Bounty', 'Year'] = 1935

for index, row in df.iterrows():
    if pd.isna(row["name"]):
        print(f"Row {index} contains NaN values:")
        print(row)
        print()

In [1022]:
df

,name,url,winner,budget,Year,distributor
0,Wings,https://en.wikipedia.org/wiki/Wings_(1927_film),True,2000000,1927,Paramount Famous Lasky Corporation
1,Rocky,https://en.wikipedia.org/wiki/Rocky,True,860000,1976,United Artists
2,Shakespeare in Love,https://en.wikipedia.org/wiki/Shakespeare_in_Love,True,25000000,1998,Miramax Films
3,Titanic,https://en.wikipedia.org/wiki/Titanic_(1997_film),True,200000000,1997,20th Century Fox
4,The English Patient,https://en.wikipedia.org/wiki/The_English_Pati...,True,27000000,1996,Miramax Films
...,...,...,...,...,...,...
586,The Hustler,https://en.wikipedia.org/wiki/The_Hustler,False,2100000,1961,20th Century-Fox
587,Judgment at Nuremberg,https://en.wikipedia.org/wiki/Judgment_at_Nure...,False,3000000,1961,United Artists
588,The Longest Day,https://en.wikipedia.org/wiki/The_Longest_Day_...,False,7750000,1962,20th Century Fox
589,The Music Man,https://en.wikipedia.org/wiki/The_Music_Man_(1...,False,NaN,1962,Warner Bros. Pictures


In [1023]:
df.loc[df['name'] == 'Star Wars', 'name'] = "Star Wars: Episode IV - A New Hope"
df.loc[df['name'] == 'Good Night, and Good Luck', 'name'] = "Good Night, and Good Luck."
data.loc[data['Title'] == 'The Postman (Il Postino) - Behind the scenes', 'Title'] = "The Postman (Il Postino)"
df.loc[df['name'] == 'Grand Illusion', 'name'] = "The Grand Illusion"


In [1024]:
# df.to_csv('film_budget_data.csv', index=False)

In [1025]:
data = pd.read_json('movies_list (1).json')

In [1026]:
# data.replace("N/A", "NaN", inplace=True)
# data.replace("nan", "NaN", inplace=True)
# data.isna().sum()

In [1027]:
sum(data["BoxOffice"] == "N/A")

436

In [1028]:
df

,name,url,winner,budget,Year,distributor
0,Wings,https://en.wikipedia.org/wiki/Wings_(1927_film),True,2000000,1927,Paramount Famous Lasky Corporation
1,Rocky,https://en.wikipedia.org/wiki/Rocky,True,860000,1976,United Artists
2,Shakespeare in Love,https://en.wikipedia.org/wiki/Shakespeare_in_Love,True,25000000,1998,Miramax Films
3,Titanic,https://en.wikipedia.org/wiki/Titanic_(1997_film),True,200000000,1997,20th Century Fox
4,The English Patient,https://en.wikipedia.org/wiki/The_English_Pati...,True,27000000,1996,Miramax Films
...,...,...,...,...,...,...
586,The Hustler,https://en.wikipedia.org/wiki/The_Hustler,False,2100000,1961,20th Century-Fox
587,Judgment at Nuremberg,https://en.wikipedia.org/wiki/Judgment_at_Nure...,False,3000000,1961,United Artists
588,The Longest Day,https://en.wikipedia.org/wiki/The_Longest_Day_...,False,7750000,1962,20th Century Fox
589,The Music Man,https://en.wikipedia.org/wiki/The_Music_Man_(1...,False,NaN,1962,Warner Bros. Pictures


In [1029]:
# print(pd.merge(df, data, on="Title"))

In [1030]:
data["Title"].nunique()

559

In [1031]:
print(df["name"].nunique())
df = df.rename(columns = {"name": "Title"})

582


In [1032]:
df['Year'] = df['Year'].astype(float)
df.sort_values(by="Year", ascending=False)

,Title,url,winner,budget,Year,distributor
294,Women Talking,https://en.wikipedia.org/wiki/Women_Talking_(f...,False,NaN,2022.0,United Artists Releasing
293,Triangle of Sadness,https://en.wikipedia.org/wiki/Triangle_of_Sadness,False,15600000,2022.0,SF Studios
292,Top Gun: Maverick,https://en.wikipedia.org/wiki/Top_Gun:_Maverick,False,170000000,2022.0,Paramount Pictures
291,Tár,https://en.wikipedia.org/wiki/T%C3%A1r,False,25000000,2022.0,Focus Features
290,The Fabelmans,https://en.wikipedia.org/wiki/The_Fabelmans,False,40000000,2022.0,Universal Pictures
...,...,...,...,...,...,...
397,The Patriot,https://en.wikipedia.org/wiki/The_Patriot_(192...,False,1000000,1928.0,Paramount Pictures
392,7th Heaven,https://en.wikipedia.org/wiki/7th_Heaven_(1927...,False,1300000,1927.0,Fox Film Corporation
0,Wings,https://en.wikipedia.org/wiki/Wings_(1927_film),True,2000000,1927.0,Paramount Famous Lasky Corporation
332,Kings Row,https://en.wikipedia.org/wiki/Kings_Row,False,1081698,NaN,Warner Bros.


In [1065]:

df['distributor'] = df['distributor'].replace('Paramount Famous Lasky Corporation', 'Paramount Pictures')
df['distributor'] = df['distributor'].replace('Warner Bros. Pictures', 'Warner Bros.')
df['distributor'] = df['distributor'].replace('Warner Bros.-Seven Arts', 'Warner Bros.')
df['distributor'] = df['distributor'].replace('Paramount Famous Lasky Corp.', 'Paramount Pictures')
df['distributor'] = df['distributor'].replace('Warner Bros. France', 'Warner Bros.')
df['distributor'] = df['distributor'].replace('Columbia TriStar Film Distributors International', 'Columbia Pictures')
df['distributor'] = df['distributor'].replace('Columbia', 'Columbia Pictures')
df['distributor'] = df['distributor'].replace('Walt Disney StudiosMotion Pictures', 'Walt Disney Studios Motion Pictures')
df['distributor'] = df['distributor'].replace('Buena Vista Pictures Distribution', 'Walt Disney Studios Motion Pictures')
df['distributor'] = df['distributor'].replace('Buena Vista Pictures Distribution, Inc.', 'Walt Disney Studios Motion Pictures')
df['distributor'] = df['distributor'].replace('Columbia TriStar Film Distributors International', 'Columbia Pictures')
df['distributor'] = df['distributor'].replace('Warner Bros. Pictures, Inc.', 'Warner Bros.')
df['distributor'] = df['distributor'].replace('Warner Bros.-Seven Arts', 'Warner Bros.')
df['distributor'] = df['distributor'].replace('Buena Vista Distribution Company, Inc.', 'Walt Disney Studios Motion Pictures')
df['distributor'] = df['distributor'].replace('MGM/UA Communications Co.', 'Metro-Goldwyn-Mayer')
df['distributor'] = df['distributor'].replace('Columbia Pictures Corporation', 'Columbia Pictures')
df['distributor'] = df['distributor'].replace("Loew's Inc.", 'Metro-Goldwyn-Mayer')
df['distributor'] = df['distributor'].replace('Columbia Pictures Corporation', 'Columbia Pictures')




print(df['distributor'].value_counts()[:30])



distributor
Warner Bros.                              79
Paramount Pictures                        61
Columbia Pictures                         49
United Artists                            43
20th Century Fox                          42
Metro-Goldwyn-Mayer                       38
Universal Pictures                        28
RKO Radio Pictures                        17
Loew's, Inc.                              13
Focus Features                            13
Fox Searchlight Pictures                  13
20th Century-Fox                          12
Walt Disney Studios Motion Pictures       12
Sony Pictures Releasing                   11
Miramax Films                             10
[1]                                        9
Fox Film Corporation                       7
Netflix                                    7
Sony Pictures Classics                     6
Orion Pictures                             6
A24                                        4
Twentieth Century-Fox Film Corporation     

In [1066]:
# combodata = pd.merge(df, data, on="Title")
# combodata = pd.DataFrame(combodata)
# combodata.to_markdown()
merged_df = pd.merge(df, data, on=['Year', 'Title'], how='left')

# Display the merged dataframe
# merged_df["Title"].nunique()

In [1067]:
combodata["Title"].nunique()

552

In [1068]:
trimmeddata = merged_df.drop_duplicates(subset="url")

In [1069]:
# data.drop_duplicates(subset="imdbID")
# for year in df:
#     year["Year_x"] = int(year[4])
# # df["Year_x"] = df["Year_x"]
# df["Year_x"].describe()

In [1070]:
# desired_movie = next((movie for movie in film_data_final if movie["name"] == "Hollywood Revue"), None)

# if desired_movie:
#     desired_movie["year"] = 1929
#     print(desired_movie)
# else:
#     print("Movie not found.")


In [1071]:
trimmeddata = trimmeddata[trimmeddata["Year"] <= 2020]
trimmeddata

,Title,url,winner,budget,Year,distributor,Rated,Released,Runtime,Genre,...,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,Error
0,Wings,https://en.wikipedia.org/wiki/Wings_(1927_film),True,2000000,1927.0,Paramount Pictures,PG-13,05 Jan 1929,144 min,"Drama, Romance, War",...,7.6,"14,066",tt0018578,movie,20 Feb 2012,N/A,N/A,N/A,True,NaN
2,Rocky,https://en.wikipedia.org/wiki/Rocky,True,860000,1976.0,United Artists,PG,03 Dec 1976,120 min,"Drama, Sport",...,8.1,"612,815",tt0075148,movie,31 Oct 2015,"$117,235,147",N/A,N/A,True,NaN
4,Shakespeare in Love,https://en.wikipedia.org/wiki/Shakespeare_in_Love,True,25000000,1998.0,Miramax Films,R,08 Jan 1999,123 min,"Comedy, Drama, History",...,7.1,"231,643",tt0138097,movie,21 Apr 2016,"$100,317,794",N/A,N/A,True,NaN
6,Titanic,https://en.wikipedia.org/wiki/Titanic_(1997_film),True,200000000,1997.0,20th Century Fox,PG-13,19 Dec 1997,194 min,"Drama, Romance",...,7.9,"1,252,252",tt0120338,movie,01 Jun 2014,"$674,292,608",N/A,N/A,True,NaN
8,The English Patient,https://en.wikipedia.org/wiki/The_English_Pati...,True,27000000,1996.0,Miramax Films,R,06 Dec 1996,162 min,"Drama, Romance, War",...,7.4,"196,938",tt0116209,movie,30 Nov 2016,"$78,676,425",N/A,N/A,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140,The Hustler,https://en.wikipedia.org/wiki/The_Hustler,False,2100000,1961.0,20th Century-Fox,Not Rated,25 Sep 1961,134 min,"Drama, Sport",...,8.0,"85,488",tt0054997,movie,25 Nov 2015,N/A,N/A,N/A,True,NaN
1142,Judgment at Nuremberg,https://en.wikipedia.org/wiki/Judgment_at_Nure...,False,3000000,1961.0,United Artists,Approved,18 Dec 1961,179 min,"Drama, War",...,8.3,"82,508",tt0055031,movie,N/A,N/A,N/A,N/A,True,NaN
1144,The Longest Day,https://en.wikipedia.org/wiki/The_Longest_Day_...,False,7750000,1962.0,20th Century Fox,G,04 Oct 1962,178 min,"Action, Drama, History",...,7.7,"58,214",tt0056197,movie,01 Mar 2013,N/A,N/A,N/A,True,NaN
1146,The Music Man,https://en.wikipedia.org/wiki/The_Music_Man_(1...,False,NaN,1962.0,Warner Bros.,G,19 Jun 1962,151 min,"Comedy, Family, Musical",...,7.7,"18,727",tt0056262,movie,19 Apr 2016,N/A,N/A,N/A,True,NaN


In [1072]:
for index, row in trimmeddata.iterrows():
    if pd.isna(row["imdbID"]):
        print(row["Title"])

Precious: Based on the Novel 'Push' by Sapphire
Lincoln
The Postman (Il Postino)
It's a Wonderful Life
The Ox-Bow Incident
Hollywood Revue
Cries and Whispers
Reds
Sunset Boulevard
Room at the Top


In [1073]:
titles_with_star = data[data['Title'].str.contains('Lincoln', case=False, na=False)]
print(titles_with_star)

        Title    Year Rated Released Runtime                   Genre  \
499   Lincoln  2011.0   N/A      N/A   4 min  Short, Comedy, Fantasy   
1070  Lincoln  2011.0   N/A      N/A   4 min  Short, Comedy, Fantasy   

            Director          Writer  \
499   Nathan Hoffman  Nathan Hoffman   
1070  Nathan Hoffman  Nathan Hoffman   

                                                 Actors  \
499   Kevin La Hood Burns, Jared Tell, Chris Dorhout...   
1070  Kevin La Hood Burns, Jared Tell, Chris Dorhout...   

                                                   Plot  ... imdbRating  \
499   When Kevin forgets about an essay that's due, ...  ...        N/A   
1070  When Kevin forgets about an essay that's due, ...  ...        N/A   

     imdbVotes     imdbID   Type  DVD BoxOffice Production Website Response  \
499        N/A  tt1834849  movie  N/A       N/A        N/A     N/A     True   
1070       N/A  tt1834849  movie  N/A       N/A        N/A     N/A     True   

     Error  
499    

In [1074]:
# winners = df[df['Winner']]
for index, row in trimmeddata.iterrows():
    box = str(row['BoxOffice'])
    # print(box)
    box = re.sub(r',', '', box)
    box = re.sub(r'\$', '', box)
    if box == "N/A":
        box = "nan"
    elif box != "nan":
        box = int(box)
    # print(box)
    row["BoxOffice"] = box
    trimmeddata.loc[index, "BoxOffice"] = box
# Group by 'Year' and calculate rank for 'BoxOffice' and 'Budget'
# trimmeddata['BoxOffice_Rank'] = trimmeddata.groupby('Year')['BoxOffice'].rank(ascending=False, method="min")
# trimmeddata['Budget_Rank'] = trimmeddata.groupby('Year')['budget'].rank(ascending=False, method="min")

In [1075]:
trimmeddata[trimmeddata["Year"]==2000]

,Title,url,winner,budget,Year,distributor,Rated,Released,Runtime,Genre,...,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,Error
48,Gladiator,https://en.wikipedia.org/wiki/Gladiator_(2000_...,True,103000000,2000.0,DreamWorks Distribution,R,05 May 2000,155 min,"Action, Adventure, Drama",...,8.5,"1,573,055",tt0172495,movie,15 Jun 2011,187705427,N/A,N/A,True,NaN
439,Traffic,https://en.wikipedia.org/wiki/Traffic_(2000_film),False,48000000,2000.0,USA Films,R,05 Jan 2001,147 min,"Crime, Drama, Thriller",...,7.6,"216,641",tt0181865,movie,10 Sep 2015,124115725,N/A,N/A,True,NaN
441,Erin Brockovich,https://en.wikipedia.org/wiki/Erin_Brockovich_...,False,52000000,2000.0,Columbia Pictures,R,17 Mar 2000,131 min,"Biography, Drama",...,7.4,"212,313",tt0195685,movie,25 Feb 2014,125595205,N/A,N/A,True,NaN
443,"Crouching Tiger, Hidden Dragon","https://en.wikipedia.org/wiki/Crouching_Tiger,...",False,17000000,2000.0,Sony Pictures Classics,PG-13,12 Jan 2001,120 min,"Action, Adventure, Drama",...,7.9,"278,303",tt0190332,movie,16 Apr 2012,128530421,N/A,N/A,True,NaN
461,Chocolat,https://en.wikipedia.org/wiki/Chocolat_(2000_f...,False,25000000,2000.0,Miramax Films,PG-13,19 Jan 2001,121 min,"Drama, Romance",...,7.3,"198,526",tt0241303,movie,14 Apr 2016,71509363,N/A,N/A,True,NaN


In [1076]:
trimmeddata["BoxOffice"].describe()
for index, row in trimmeddata.iterrows():
    if row["BoxOffice"] == "nan":
        trimmeddata.loc[index, "BoxOffice"] = np.nan
    elif pd.isna(row["BoxOffice"])==False:
        trimmeddata.loc[index, "BoxOffice"] = int(row["BoxOffice"])

trimmeddata["budget"].describe()
for index, row in trimmeddata.iterrows():
    if row["budget"] == "nan":
        trimmeddata.loc[index, "budget"] = np.nan
    elif pd.isna(row["budget"])==False:
        trimmeddata.loc[index, "budget"] = int(row["budget"])

In [1077]:
trimmeddata['BoxOffice_Rank'] = trimmeddata.groupby('Year')['BoxOffice'].rank(ascending=False, method="min")
trimmeddata['Budget_Rank'] = trimmeddata.groupby('Year')['budget'].rank(ascending=False, method="min")

In [1078]:
trimmeddata[trimmeddata["Year"] == 2000]
# trimmeddata = pd.DataFrame(trimmeddata)

,Title,url,winner,budget,Year,distributor,Rated,Released,Runtime,Genre,...,imdbID,Type,DVD,BoxOffice,Production,Website,Response,Error,BoxOffice_Rank,Budget_Rank
48,Gladiator,https://en.wikipedia.org/wiki/Gladiator_(2000_...,True,103000000,2000.0,DreamWorks Distribution,R,05 May 2000,155 min,"Action, Adventure, Drama",...,tt0172495,movie,15 Jun 2011,187705427,N/A,N/A,True,NaN,1.0,1.0
439,Traffic,https://en.wikipedia.org/wiki/Traffic_(2000_film),False,48000000,2000.0,USA Films,R,05 Jan 2001,147 min,"Crime, Drama, Thriller",...,tt0181865,movie,10 Sep 2015,124115725,N/A,N/A,True,NaN,4.0,3.0
441,Erin Brockovich,https://en.wikipedia.org/wiki/Erin_Brockovich_...,False,52000000,2000.0,Columbia Pictures,R,17 Mar 2000,131 min,"Biography, Drama",...,tt0195685,movie,25 Feb 2014,125595205,N/A,N/A,True,NaN,3.0,2.0
443,"Crouching Tiger, Hidden Dragon","https://en.wikipedia.org/wiki/Crouching_Tiger,...",False,17000000,2000.0,Sony Pictures Classics,PG-13,12 Jan 2001,120 min,"Action, Adventure, Drama",...,tt0190332,movie,16 Apr 2012,128530421,N/A,N/A,True,NaN,2.0,5.0
461,Chocolat,https://en.wikipedia.org/wiki/Chocolat_(2000_f...,False,25000000,2000.0,Miramax Films,PG-13,19 Jan 2001,121 min,"Drama, Romance",...,tt0241303,movie,14 Apr 2016,71509363,N/A,N/A,True,NaN,5.0,4.0


In [1079]:
trimmeddata

,Title,url,winner,budget,Year,distributor,Rated,Released,Runtime,Genre,...,imdbID,Type,DVD,BoxOffice,Production,Website,Response,Error,BoxOffice_Rank,Budget_Rank
0,Wings,https://en.wikipedia.org/wiki/Wings_(1927_film),True,2000000,1927.0,Paramount Pictures,PG-13,05 Jan 1929,144 min,"Drama, Romance, War",...,tt0018578,movie,20 Feb 2012,NaN,N/A,N/A,True,NaN,NaN,1.0
2,Rocky,https://en.wikipedia.org/wiki/Rocky,True,860000,1976.0,United Artists,PG,03 Dec 1976,120 min,"Drama, Sport",...,tt0075148,movie,31 Oct 2015,117235147,N/A,N/A,True,NaN,1.0,5.0
4,Shakespeare in Love,https://en.wikipedia.org/wiki/Shakespeare_in_Love,True,25000000,1998.0,Miramax Films,R,08 Jan 1999,123 min,"Comedy, Drama, History",...,tt0138097,movie,21 Apr 2016,100317794,N/A,N/A,True,NaN,2.0,4.0
6,Titanic,https://en.wikipedia.org/wiki/Titanic_(1997_film),True,200000000,1997.0,20th Century Fox,PG-13,19 Dec 1997,194 min,"Drama, Romance",...,tt0120338,movie,01 Jun 2014,674292608,N/A,N/A,True,NaN,1.0,1.0
8,The English Patient,https://en.wikipedia.org/wiki/The_English_Pati...,True,27000000,1996.0,Miramax Films,R,06 Dec 1996,162 min,"Drama, Romance, War",...,tt0116209,movie,30 Nov 2016,78676425,N/A,N/A,True,NaN,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140,The Hustler,https://en.wikipedia.org/wiki/The_Hustler,False,2100000,1961.0,20th Century-Fox,Not Rated,25 Sep 1961,134 min,"Drama, Sport",...,tt0054997,movie,25 Nov 2015,NaN,N/A,N/A,True,NaN,NaN,4.0
1142,Judgment at Nuremberg,https://en.wikipedia.org/wiki/Judgment_at_Nure...,False,3000000,1961.0,United Artists,Approved,18 Dec 1961,179 min,"Drama, War",...,tt0055031,movie,N/A,NaN,N/A,N/A,True,NaN,NaN,3.0
1144,The Longest Day,https://en.wikipedia.org/wiki/The_Longest_Day_...,False,7750000,1962.0,20th Century Fox,G,04 Oct 1962,178 min,"Action, Drama, History",...,tt0056197,movie,01 Mar 2013,NaN,N/A,N/A,True,NaN,NaN,3.0
1146,The Music Man,https://en.wikipedia.org/wiki/The_Music_Man_(1...,False,NaN,1962.0,Warner Bros.,G,19 Jun 1962,151 min,"Comedy, Family, Musical",...,tt0056262,movie,19 Apr 2016,NaN,N/A,N/A,True,NaN,NaN,NaN


In [1080]:
trimmeddata=pd.DataFrame(trimmeddata)
trimmeddata.to_json("cleanerdata.json", orient="records", lines=True)

In [1081]:
# nonans = trimmeddata.fillna("NA", inplace=True)
# print(trimmeddata.shape)
# listoffilms = trimmeddata.to_dict(orient="records")
# json_string = json.dumps(listoffilms)
# formatted_string = ""
# for char in json_string:
#     if char == '{':
#         formatted_string += char + ","
#     elif char == '}':
#         formatted_string += char + ","
#     else:
#         formatted_string += char

# # Add square brackets around the whole thing
# final_result = "[" + formatted_string[:-1] + "]"
# print(listoffilms)
# print(listoffilms.shape)
# with open("cleanerdata.json", "w") as json_file:
#     json.dump(listoffilms, json_file)

In [1096]:
trimmeddata.replace({np.nan: None, pd.NaT: None}, inplace=True)


In [1097]:
data_list = trimmeddata.to_dict(orient="records")

# Specify the file path for the JSON file
json_file_path = "cleanestdata.json"

# Write the data to the JSON file
with open(json_file_path, "w") as json_file:
    json.dump(data_list, json_file, indent=2)